In [14]:
pip show azure-ai-ml

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Connect to workspace
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential doesn't work
    credential = InteractiveBrowserCredential()

In [ ]:
# Get handle to workspace
ml_client = MLClient.from_config(credential=credential)

In [ ]:
# Prepare data
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import Input

# Creates a dataset based on the files in the local data folder
my_training_data_input = Input(type=AssetTypes.URI_FILE, path="azureml:tickets-data:1")

In [ ]:
# Configure automated machine learning job
from azure.ai.ml import automl

# Configure the classification job
classification_job = automl.classification(
    compute="aml-cluster",
    experiment_name="auto-ml-class-dev",
    training_data=my_training_data_input,
    target_column="label",
    primary_metric="accuracy",
    n_cross_validations=5,
)

# Set the limits (optional)
classification_job.set_limits(
    timeout_minutes=60,
    trial_timeout_minutes=20,
    max_trials=5,
    enable_early_termination=True,
)

# Set the training properties (optional)
classification_job.set_training(
    blocked_training_algorithms=["LogisticRegression"],
    enable_onnx_compatible_models=True,
)

In [ ]:
# Run an automated machine learning job
# Submit the AutoML job
returned_job = ml_client.jobs.create_or_update(
    classification_job
)

# Submit the job to the backend
aml_url = returned_job.studio_url
print("Monitor your job at: ", aml_url)